In [1]:
import pandas as pd
import numpy as np

In [30]:
df=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')


In [31]:
import re
import html
import nltk

# Download required NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # 1. Decode HTML entities
    text = html.unescape(text)

    # 2. Lowercase
    text = text.lower()

    # 3. Remove unwanted symbols/punctuation/digits
    text = re.sub(r'["“”\'\*\`~\-=&;#\\/<>\|\[\]\(\)_¶]', ' ', text)  # symbols
    text = re.sub(r'\.{2,}', ' ', text)  # multiple dots


    # 4. Tokenize
    tokens = word_tokenize(text)


    # 6. Lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # 7. Join back to string
    return ' '.join(tokens)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [32]:
df['clean_text']=df['text'].apply(preprocess_text)

In [33]:
test['clean_text']=test['text'].apply(preprocess_text)


In [34]:
df[df.duplicated(subset=['clean_text','emotions'],keep=False)].sort_values(by='clean_text')

,id,text,anger,fear,joy,sadness,surprise,emotions,clean_text
1660,1660,", the blisters on my feet ( the wound on my la...",0,1,0,1,0,['fear' 'sadness'],", the blister on my foot the wound on my large..."
2591,2591,", the blisters on my feet ( the wound on my la...",0,1,0,1,0,['fear' 'sadness'],", the blister on my foot the wound on my large..."
4270,4270,12:00 -- making lunch while a toddler hangs on...,0,1,0,1,0,['fear' 'sadness'],12:00 making lunch while a toddler hang on my ...
3884,3884,12:00 -- making lunch while a toddler hangs on...,0,1,0,1,0,['fear' 'sadness'],12:00 making lunch while a toddler hang on my ...
2085,2085,"1991, about 8:00 a.m. in the morning on a satu...",0,1,0,0,0,['fear'],"1991 , about 8:00 a.m. in the morning on a sat..."
...,...,...,...,...,...,...,...,...,...
3415,3415,your just at work.,0,0,0,0,0,[],your just at work .
3595,3595,your mom is nuts.,1,0,0,0,0,['anger'],your mom is nut .
5495,5495,your mom is nuts.,1,0,0,0,0,['anger'],your mom is nut .
1252,1252,your story is really creepy.,0,1,0,0,1,['fear' 'surprise'],your story is really creepy .


In [35]:
df=df.drop_duplicates(subset=['clean_text', 'emotions'], keep='first').reset_index()



In [36]:

import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from collections import Counter
from tqdm import tqdm
import pandas as pd

# ---------- Reproducibility ----------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------- Settings ----------
EMOTION_COLS = ['anger', 'fear', 'joy', 'sadness', 'surprise']


X = df['clean_text'].astype(str)
Y = df[EMOTION_COLS].astype(int)

x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.20, random_state=SEED, shuffle=True)


Device: cuda


In [37]:
def build_vocab(texts, min_freq=2):
    counter = Counter()
    for t in texts:
        for tok in t.split():
            counter[tok] += 1
    vocab = {'<PAD>': 0, '<UNK>': 1}
    for word, freq in counter.items():
        if freq >= min_freq:
            vocab[word] = len(vocab)
    return vocab

vocab = build_vocab(x_train, min_freq=1)
vocab_size = len(vocab)
print("✅ Vocab size:", vocab_size)

✅ Vocab size: 6564


In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from collections import Counter
from tqdm import tqdm
import pandas as pd

# ----------------- CONFIG -----------------
EMOTION_COLS = ['anger', 'fear', 'joy', 'sadness', 'surprise']
MAX_LEN = 80           # 🔼 slightly longer for more context
MIN_FREQ = 1
BATCH_SIZE = 64        # 🔼 larger batch = better gradient estimates
EMBED_DIM = 256        # 🔼 richer word features
HIDDEN_DIM = 256       # 🔼 more capacity
NUM_LAYERS = 3         # reduced for efficiency
DROPOUT = 0.4          # slightly higher dropout for regularization
NUM_EPOCHS = 15
LEARNING_RATE = 0.07
WEIGHT_DECAY = 1e-5    # 🔥 helps avoid overfitting (L2 regularization)
GRAD_CLIP = 1.0        # 🔥 gradient clipping to stabilize RNN
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------- BUILD VOCAB -----------------
def build_vocab(texts, min_freq=2):
    counter = Counter()
    for t in texts:
        for tok in t.split():
            counter[tok] += 1
    vocab = {'<PAD>': 0, '<UNK>': 1}
    for word, freq in counter.items():
        if freq >= min_freq:
            vocab[word] = len(vocab)
    return vocab

vocab = build_vocab(x_train, min_freq=MIN_FREQ)
vocab_size = len(vocab)
print("✅ Vocab size:", vocab_size)

def text_to_ids(text, vocab):
    return [vocab.get(tok, vocab['<UNK>']) for tok in text.split()]

# ----------------- DATASET -----------------
class RNNTextDataset(Dataset):
    def __init__(self, texts, labels, vocab):
        self.texts = texts.tolist()
        self.labels = labels.values
        self.vocab = vocab

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        ids = text_to_ids(self.texts[idx], self.vocab)
        label = torch.tensor(self.labels[idx]).float()
        return torch.tensor(ids, dtype=torch.long), label

def collate_batch(batch, max_len=MAX_LEN):
    seqs, labels = zip(*batch)
    lengths = [min(len(s), max_len) for s in seqs]
    padded = torch.full((len(seqs), max_len), fill_value=vocab['<PAD>'], dtype=torch.long)
    for i, s in enumerate(seqs):
        truncated = s[:max_len]
        padded[i, :len(truncated)] = truncated
    lengths = torch.tensor(lengths, dtype=torch.long)
    labels = torch.stack(labels)
    return padded, lengths, labels

train_ds = RNNTextDataset(x_train, y_train, vocab)
val_ds   = RNNTextDataset(x_val,   y_val,   vocab)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=lambda b: collate_batch(b, MAX_LEN))
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE*2, shuffle=False, collate_fn=lambda b: collate_batch(b, MAX_LEN))

# ----------------- MODEL -----------------
class VanillaRNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_layers=1, dropout=0.3):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.rnn = nn.RNN(input_size=embed_dim,
                          hidden_size=hidden_dim,
                          num_layers=num_layers,
                          batch_first=True,
                          nonlinearity='tanh',
                          dropout=dropout if num_layers > 1 else 0.0)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, num_classes)

        # 🔥 Xavier initialization for better convergence
        for name, param in self.rnn.named_parameters():
            if "weight" in name:
                nn.init.xavier_uniform_(param)
            elif "bias" in name:
                nn.init.constant_(param, 0)

    def forward(self, input_ids, lengths):
        emb = self.embed(input_ids)
        outputs, h_n = self.rnn(emb)
        last_hidden = h_n[-1]
        out = self.dropout(last_hidden)
        logits = self.fc(out)
        return logits

model = VanillaRNNClassifier(
    vocab_size=vocab_size,
    embed_dim=EMBED_DIM,
    hidden_dim=HIDDEN_DIM,
    num_classes=len(EMOTION_COLS),
    num_layers=NUM_LAYERS,
    dropout=DROPOUT
).to(device)

# ----------------- LOSS / OPTIMIZER -----------------
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)  # 🔥 AdamW > Adam
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1)

# ----------------- TRAIN / EVAL -----------------
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    for input_ids, lengths, labels in tqdm(loader, desc="Train", leave=False):
        input_ids, lengths, labels = input_ids.to(device), lengths.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(input_ids, lengths)
        loss = criterion(logits, labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)  # 🔥 prevent exploding gradients
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    preds_list, labels_list = [], []
    with torch.no_grad():
        for input_ids, lengths, labels in loader:
            input_ids, lengths, labels = input_ids.to(device), lengths.to(device), labels.to(device)
            logits = model(input_ids, lengths)
            loss = criterion(logits, labels)
            total_loss += loss.item()
            probs = torch.sigmoid(logits).cpu().numpy()
            preds_list.append(probs)
            labels_list.append(labels.cpu().numpy())

    preds = np.vstack(preds_list)
    labels = np.vstack(labels_list)
    preds_bin = (preds >= 0.5).astype(int)
    f1 = f1_score(labels, preds_bin, average='macro', zero_division=0)
    return total_loss / len(loader), f1

# ----------------- TRAINING LOOP -----------------
best_val_f1 = 0.0
for epoch in range(1, NUM_EPOCHS + 1):
    train_loss = train_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_f1 = evaluate(model, val_loader, criterion)
    scheduler.step(val_f1)

    print(f"Epoch {epoch:02d}/{NUM_EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val F1: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), "best_vanilla_rnn.pth")
        print(f"✅ Saved best model (F1={best_val_f1:.4f})")

print("🎯 Training finished. Best Val F1:", best_val_f1)


✅ Vocab size: 6564


Epoch 01/15 | Train Loss: 1.0828 | Val Loss: 0.6487 | Val F1: 0.1489
✅ Saved best model (F1=0.1489)


Epoch 02/15 | Train Loss: 0.8568 | Val Loss: 0.8240 | Val F1: 0.2598
✅ Saved best model (F1=0.2598)


Epoch 03/15 | Train Loss: 0.8393 | Val Loss: 0.6522 | Val F1: 0.1387


Epoch 04/15 | Train Loss: 0.8935 | Val Loss: 1.0725 | Val F1: 0.2804
✅ Saved best model (F1=0.2804)


...

Epoch 11/15 | Train Loss: 0.6065 | Val Loss: 0.6735 | Val F1: 0.1705


Epoch 12/15 | Train Loss: 0.6136 | Val Loss: 0.6168 | Val F1: 0.1034


Epoch 13/15 | Train Loss: 0.5943 | Val Loss: 0.5712 | Val F1: 0.1468


Epoch 14/15 | Train Loss: 0.5921 | Val Loss: 0.5683 | Val F1: 0.1468


Epoch 15/15 | Train Loss: 0.5760 | Val Loss: 0.5600 | Val F1: 0.1468
🎯 Training finished. Best Val F1: 0.280402782084068


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import f1_score
from collections import Counter
from tqdm import tqdm

# ---------------- Hyperparameters ----------------
EMOTION_COLS = ['anger', 'fear', 'joy', 'sadness', 'surprise']
MAX_LEN = 80
MIN_FREQ = 1
BATCH_SIZE = 64
EMBED_DIM = 256
HIDDEN_DIM = 256
NUM_LAYERS = 2
DROPOUT = 0.4
NUM_EPOCHS = 50
LEARNING_RATE = 5e-4
device = "cuda" if torch.cuda.is_available() else "cpu"

# ---------------- Vocabulary ----------------
def build_vocab(texts, min_freq=2):
    counter = Counter()
    for t in texts:
        for tok in t.split():
            counter[tok] += 1
    vocab = {'<PAD>': 0, '<UNK>': 1}
    for word, freq in counter.items():
        if freq >= min_freq:
            vocab[word] = len(vocab)
    return vocab

def text_to_ids(text, vocab):
    return [vocab.get(tok, vocab['<UNK>']) for tok in text.split()]

# ---------------- Dataset ----------------
class TextDataset(Dataset):
    def __init__(self, texts, labels=None, vocab=None):
        self.texts = texts.tolist()
        self.labels = labels.values if labels is not None else None
        self.vocab = vocab

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        ids = text_to_ids(self.texts[idx], self.vocab)
        if self.labels is not None:
            label = torch.tensor(self.labels[idx]).float()
            return torch.tensor(ids, dtype=torch.long), label
        return torch.tensor(ids, dtype=torch.long)

def collate_batch(batch, max_len=MAX_LEN):
    if isinstance(batch[0], tuple):
        seqs, labels = zip(*batch)
    else:
        seqs = batch
        labels = None
    lengths = [min(len(s), max_len) for s in seqs]
    padded = torch.full((len(seqs), max_len), fill_value=vocab['<PAD>'], dtype=torch.long)
    for i, s in enumerate(seqs):
        truncated = s[:max_len]
        padded[i, :len(truncated)] = truncated
    lengths = torch.tensor(lengths, dtype=torch.long)
    if labels is not None:
        labels = torch.stack(labels)
        return padded, lengths, labels
    return padded, lengths

# ---------------- GRU Model ----------------
class GRUClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_layers=1, dropout=0.3, bidirectional=True):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.gru = nn.GRU(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
            bidirectional=bidirectional
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, num_classes)

    def forward(self, input_ids, lengths):
        emb = self.embed(input_ids)
        packed_output, h_n = self.gru(emb)
        # h_n: (num_layers * num_directions, batch, hidden)
        if self.gru.bidirectional:
            last_hidden = torch.cat((h_n[-2], h_n[-1]), dim=1)  # concat last forward + backward
        else:
            last_hidden = h_n[-1]
        out = self.dropout(last_hidden)
        logits = self.fc(out)
        return logits

# ---------------- Initialize ----------------
vocab = build_vocab(x_train, min_freq=MIN_FREQ)
vocab_size = len(vocab)
model = GRUClassifier(
    vocab_size=vocab_size,
    embed_dim=EMBED_DIM,
    hidden_dim=HIDDEN_DIM,
    num_classes=len(EMOTION_COLS),
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
    bidirectional=True
).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# ---------------- Training ----------------
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    for input_ids, lengths, labels in tqdm(loader, leave=False):
        input_ids, lengths, labels = input_ids.to(device), lengths.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(input_ids, lengths)
        loss = criterion(logits, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)  # gradient clipping
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    preds_list, labels_list = [], []
    with torch.no_grad():
        for input_ids, lengths, labels in loader:
            input_ids, lengths, labels = input_ids.to(device), lengths.to(device), labels.to(device)
            logits = model(input_ids, lengths)
            loss = criterion(logits, labels)
            total_loss += loss.item()
            probs = torch.sigmoid(logits).cpu().numpy()
            preds_list.append(probs)
            labels_list.append(labels.cpu().numpy())
    preds = np.vstack(preds_list)
    labels = np.vstack(labels_list)
    preds_bin = (preds >= 0.5).astype(int)
    f1 = f1_score(labels, preds_bin, average='macro', zero_division=0)
    return total_loss / len(loader), f1

# ---------------- Dataloaders ----------------
train_ds = TextDataset(x_train, y_train, vocab)
val_ds   = TextDataset(x_val, y_val, vocab)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE*2, shuffle=False, collate_fn=collate_batch)

# ---------------- Training Loop ----------------
best_val_f1 = 0.0
for epoch in range(1, NUM_EPOCHS + 1):
    train_loss = train_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_f1 = evaluate(model, val_loader, criterion)
    print(f"Epoch {epoch}/{NUM_EPOCHS} | Train Loss: {train_loss:.4f} | Val F1: {val_f1:.4f}")
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), "gru_multi_label.pth")
        print("✅ Saved best model (F1:", best_val_f1, ")")


Epoch 1/50 | Train Loss: 0.5753 | Val F1: 0.1589
✅ Saved best model (F1: 0.15888636741019538 )


Epoch 2/50 | Train Loss: 0.5376 | Val F1: 0.2408
✅ Saved best model (F1: 0.24082745025699032 )


Epoch 3/50 | Train Loss: 0.5065 | Val F1: 0.3186
✅ Saved best model (F1: 0.31858049909342834 )


Epoch 4/50 | Train Loss: 0.4602 | Val F1: 0.3829
✅ Saved best model (F1: 0.3829316184621093 )


Epoch 5/50 | Train Loss: 0.4051 | Val F1: 0.4066
✅ Saved best model (F1: 0.40661196481067163 )


...

Epoch 46/50 | Train Loss: 0.0086 | Val F1: 0.4469


Epoch 47/50 | Train Loss: 0.0217 | Val F1: 0.4390


Epoch 48/50 | Train Loss: 0.0141 | Val F1: 0.4616


Epoch 49/50 | Train Loss: 0.0065 | Val F1: 0.4510


Epoch 50/50 | Train Loss: 0.0037 | Val F1: 0.4534


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
from tqdm import tqdm

# --- Dataset ---
class TextDataset(Dataset):
    def __init__(self, texts, labels, vocab, max_len=80):
        self.texts = texts
        self.labels = labels.values
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens = self.texts.iloc[idx].split()
        # numericalize + pad
        ids = [self.vocab.get(w, self.vocab["<unk>"]) for w in tokens[:self.max_len]]
        ids += [self.vocab["<pad>"]] * (self.max_len - len(ids))
        return torch.tensor(ids), torch.tensor(self.labels[idx], dtype=torch.float32)

# --- Model ---
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_layers=2, dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True  # bidirectional LSTM
        )
        self.layer_norm = nn.LayerNorm(hidden_dim * 2)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, x):
        emb = self.embedding(x)  # (batch, seq, embed_dim)
        out, (h_n, c_n) = self.lstm(emb)  # h_n: (num_layers*2, batch, hidden)
        last_hidden = torch.cat((h_n[-2], h_n[-1]), dim=1)  # concat forward + backward
        out = self.layer_norm(last_hidden)
        out = self.dropout(out)
        logits = self.fc(out)
        return logits

# --- Training Utilities ---
def train_model(model, train_loader, val_loader, epochs=10, lr=1e-3, device='cuda'):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    best_f1 = 0.0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for x_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            logits = model(x_batch)
            loss = criterion(logits, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        # --- Validation ---
        model.eval()
        val_preds, val_labels = [], []
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                x_batch = x_batch.to(device)
                logits = model(x_batch)
                preds = torch.sigmoid(logits).cpu().numpy() > 0.5
                val_preds.extend(preds)
                val_labels.extend(y_batch.numpy())

        f1 = f1_score(val_labels, val_preds, average='macro')
        print(f"Epoch {epoch+1:02d} | Train Loss: {avg_train_loss:.4f} | Val F1: {f1:.4f}")

        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), "best_lstm_model.pt")
            print(f"✅ Saved best model (F1={f1:.4f})")

# --- Vocabulary Builder ---
def build_vocab(texts, min_freq=2):
    from collections import Counter
    counter = Counter()
    for text in texts:
        counter.update(text.split())

    vocab = {"<pad>": 0, "<unk>": 1}
    for word, freq in counter.items():
        if freq >= min_freq:
            vocab[word] = len(vocab)
    return vocab


In [ ]:
# 1. Build vocab
vocab = build_vocab(x_train, min_freq=2)
print("✅ Vocab size:", len(vocab))

# 2. Prepare datasets
train_dataset = TextDataset(x_train, y_train, vocab)
val_dataset = TextDataset(x_val, y_val, vocab)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

# 3. Initialize and train
device = "cuda" if torch.cuda.is_available() else "cpu"
model = LSTMClassifier(
    vocab_size=len(vocab),
    embed_dim=128,
    hidden_dim=128,
    num_classes=y_train.shape[1],
    num_layers=2,
    dropout=0.3
).to(device)

train_model(model, train_loader, val_loader, epochs=20, lr=1e-3, device=device)


✅ Vocab size: 3043


Epoch 1/20: 100%|██████████| 63/63 [00:00<00:00, 64.74it/s]


Epoch 01 | Train Loss: 0.5834 | Val F1: 0.1358
✅ Saved best model (F1=0.1358)


Epoch 2/20: 100%|██████████| 63/63 [00:00<00:00, 79.20it/s]


Epoch 02 | Train Loss: 0.5515 | Val F1: 0.2658
✅ Saved best model (F1=0.2658)


...

Epoch 18 | Train Loss: 0.0390 | Val F1: 0.4509


Epoch 19/20: 100%|██████████| 63/63 [00:00<00:00, 73.74it/s]


Epoch 19 | Train Loss: 0.0299 | Val F1: 0.4590
✅ Saved best model (F1=0.4590)


Epoch 20/20: 100%|██████████| 63/63 [00:00<00:00, 69.35it/s]


Epoch 20 | Train Loss: 0.0273 | Val F1: 0.4517


# GRU + ATTENTion

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import f1_score
from collections import Counter
from tqdm import tqdm

# ---------------- Hyperparameters ----------------
EMOTION_COLS = ['anger', 'fear', 'joy', 'sadness', 'surprise']
MAX_LEN = 80
MIN_FREQ = 1
BATCH_SIZE = 64
EMBED_DIM = 256
HIDDEN_DIM = 256
NUM_LAYERS = 2
DROPOUT = 0.4
NUM_EPOCHS = 50
LEARNING_RATE = 5e-4
device = "cuda" if torch.cuda.is_available() else "cpu"

# ---------------- Vocabulary ----------------
def build_vocab(texts, min_freq=2):
    counter = Counter()
    for t in texts:
        for tok in t.split():
            counter[tok] += 1
    vocab = {'<PAD>': 0, '<UNK>': 1}
    for word, freq in counter.items():
        if freq >= min_freq:
            vocab[word] = len(vocab)
    return vocab

def text_to_ids(text, vocab):
    return [vocab.get(tok, vocab['<UNK>']) for tok in text.split()]

# ---------------- Dataset ----------------
class TextDataset(Dataset):
    def __init__(self, texts, labels=None, vocab=None):
        self.texts = texts.tolist()
        self.labels = labels.values if labels is not None else None
        self.vocab = vocab

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        ids = text_to_ids(self.texts[idx], self.vocab)
        if self.labels is not None:
            label = torch.tensor(self.labels[idx]).float()
            return torch.tensor(ids, dtype=torch.long), label
        return torch.tensor(ids, dtype=torch.long)

def collate_batch(batch, max_len=MAX_LEN):
    if isinstance(batch[0], tuple):
        seqs, labels = zip(*batch)
    else:
        seqs = batch
        labels = None
    lengths = [min(len(s), max_len) for s in seqs]
    padded = torch.full((len(seqs), max_len), fill_value=vocab['<PAD>'], dtype=torch.long)
    for i, s in enumerate(seqs):
        truncated = s[:max_len]
        padded[i, :len(truncated)] = truncated
    lengths = torch.tensor(lengths, dtype=torch.long)
    if labels is not None:
        labels = torch.stack(labels)
        return padded, lengths, labels
    return padded, lengths

# ---------------- GRU + Attention Model ----------------
class GRUAttentionClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_layers=1, dropout=0.3, bidirectional=True):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.gru = nn.GRU(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
            bidirectional=bidirectional
        )
        self.dropout = nn.Dropout(dropout)
        self.attention = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, num_classes)

    def forward(self, input_ids, lengths):
        emb = self.embed(input_ids)
        packed_output, h_n = self.gru(emb)  # outputs: (batch, seq_len, hidden*directions)

        # ---------------- Attention ----------------
        attn_weights = self.attention(packed_output).squeeze(-1)  # (batch, seq_len)
        mask = (input_ids != 0)  # ignore PAD tokens
        attn_weights[~mask] = float('-inf')
        attn_scores = torch.softmax(attn_weights, dim=1).unsqueeze(-1)  # (batch, seq_len, 1)
        weighted_output = (packed_output * attn_scores).sum(dim=1)      # (batch, hidden*directions)

        out = self.dropout(weighted_output)
        logits = self.fc(out)
        return logits

# ---------------- Initialize ----------------
vocab = build_vocab(x_train, min_freq=MIN_FREQ)
vocab_size = len(vocab)
model = GRUAttentionClassifier(
    vocab_size=vocab_size,
    embed_dim=EMBED_DIM,
    hidden_dim=HIDDEN_DIM,
    num_classes=len(EMOTION_COLS),
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
    bidirectional=True
).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# ---------------- Train / Eval ----------------
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    for input_ids, lengths, labels in tqdm(loader, leave=False):
        input_ids, lengths, labels = input_ids.to(device), lengths.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(input_ids, lengths)
        loss = criterion(logits, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    preds_list, labels_list = [], []
    with torch.no_grad():
        for input_ids, lengths, labels in loader:
            input_ids, lengths, labels = input_ids.to(device), lengths.to(device), labels.to(device)
            logits = model(input_ids, lengths)
            loss = criterion(logits, labels)
            total_loss += loss.item()
            probs = torch.sigmoid(logits).cpu().numpy()
            preds_list.append(probs)
            labels_list.append(labels.cpu().numpy())
    preds = np.vstack(preds_list)
    labels = np.vstack(labels_list)
    preds_bin = (preds >= 0.5).astype(int)
    f1 = f1_score(labels, preds_bin, average='macro', zero_division=0)
    return total_loss / len(loader), f1

# ---------------- Dataloaders ----------------
train_ds = TextDataset(x_train, y_train, vocab)
val_ds   = TextDataset(x_val, y_val, vocab)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE*2, shuffle=False, collate_fn=collate_batch)

# ---------------- Training Loop ----------------
best_val_f1 = 0.0
for epoch in range(1, NUM_EPOCHS + 1):
    train_loss = train_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_f1 = evaluate(model, val_loader, criterion)
    print(f"Epoch {epoch}/{NUM_EPOCHS} | Train Loss: {train_loss:.4f} | Val F1: {val_f1:.4f}")
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), "gru_attention_multi_label.pth")
        print("✅ Saved best model (F1:", best_val_f1, ")")


Epoch 1/50 | Train Loss: 0.5637 | Val F1: 0.2276
✅ Saved best model (F1: 0.22757210377278198 )


Epoch 2/50 | Train Loss: 0.5257 | Val F1: 0.3147
✅ Saved best model (F1: 0.31468548803118307 )


Epoch 3/50 | Train Loss: 0.4823 | Val F1: 0.3633
✅ Saved best model (F1: 0.3633344476727465 )


Epoch 4/50 | Train Loss: 0.4197 | Val F1: 0.4408
✅ Saved best model (F1: 0.4407900749073555 )


Epoch 5/50 | Train Loss: 0.3502 | Val F1: 0.4537
✅ Saved best model (F1: 0.4537360456591936 )


...

Epoch 46/50 | Train Loss: 0.0015 | Val F1: 0.4593


Epoch 47/50 | Train Loss: 0.0015 | Val F1: 0.4589


Epoch 48/50 | Train Loss: 0.0015 | Val F1: 0.4560


Epoch 49/50 | Train Loss: 0.0011 | Val F1: 0.4575


Epoch 50/50 | Train Loss: 0.0010 | Val F1: 0.4576


# multihead Attention


In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import f1_score

# ------------------------------------------------------------------
# 🔹 Model Definition
# ------------------------------------------------------------------
class GRU_AttentionModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_layers=2, dropout=0.4):
        super(GRU_AttentionModel, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.embedding_dropout = nn.Dropout(0.4)

        self.gru = nn.GRU(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )

        # Multi-head self-attention
        self.multihead_attn = nn.MultiheadAttention(
            embed_dim=hidden_dim * 2,  # bidirectional GRU
            num_heads=4,
            dropout=0.3,
            batch_first=True
        )

        self.layer_norm = nn.LayerNorm(hidden_dim * 2)

        # MLP classifier head
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x, lengths=None):
        emb = self.embedding_dropout(self.embedding(x))  # (batch, seq, emb)
        packed_output, _ = self.gru(emb)

        # Multi-head self-attention (Q=K=V)
        attn_output, _ = self.multihead_attn(packed_output, packed_output, packed_output)
        attn_output = self.layer_norm(attn_output + packed_output)

        # Mean pooling over time dimension
        mean_output = attn_output.mean(dim=1)

        logits = self.fc(mean_output)
        return logits


# ------------------------------------------------------------------
# 🔹 Dataset Class
# ------------------------------------------------------------------
class TextDataset(Dataset):
    def __init__(self, X, y, vocab, max_len):
        self.X = X
        self.y = y.values
        self.vocab = vocab
        self.max_len = max_len

    def text_to_seq(self, text):
        tokens = text.split()
        seq = [self.vocab.get(tok, self.vocab["<UNK>"]) for tok in tokens[:self.max_len]]
        seq += [self.vocab["<PAD>"]] * (self.max_len - len(seq))
        return torch.tensor(seq, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.text_to_seq(self.X.iloc[idx]), torch.tensor(self.y[idx], dtype=torch.float32)


# ------------------------------------------------------------------
# 🔹 Training Utilities
# ------------------------------------------------------------------
def train_epoch(model, loader, optimizer, criterion, device, scheduler=None, clip=5):
    model.train()
    total_loss = 0
    for X_batch, y_batch in tqdm(loader, desc="Training", leave=False):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)
        optimizer.step()
        if scheduler:
            scheduler.step()

        total_loss += loss.item()

    return total_loss / len(loader)


def eval_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    preds, targets = [], []
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            logits = model(X_batch)
            loss = criterion(logits, y_batch)
            total_loss += loss.item()

            preds.append(torch.sigmoid(logits).cpu())
            targets.append(y_batch.cpu())

    preds = torch.cat(preds)
    targets = torch.cat(targets)
    f1 = f1_score(targets.numpy() > 0.5, preds.numpy() > 0.5, average="macro")
    return total_loss / len(loader), f1


# ------------------------------------------------------------------
# 🔹 Training Script
# ------------------------------------------------------------------
def train_model(vocab, x_train, y_train, x_val, y_val, num_classes):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    MAX_LEN = 100
    EMBED_DIM = 256
    HIDDEN_DIM = 256
    NUM_LAYERS = 2
    DROPOUT = 0.4
    BATCH_SIZE = 64
    EPOCHS = 50
    LEARNING_RATE = 5e-4

    train_ds = TextDataset(x_train, y_train, vocab, MAX_LEN)
    val_ds = TextDataset(x_val, y_val, vocab, MAX_LEN)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

    model = GRU_AttentionModel(len(vocab), EMBED_DIM, HIDDEN_DIM, num_classes, NUM_LAYERS, DROPOUT).to(device)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
    criterion = nn.BCEWithLogitsLoss()
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=1e-3, epochs=EPOCHS, steps_per_epoch=len(train_loader)
    )

    best_f1 = 0.0
    for epoch in range(1, EPOCHS + 1):
        train_loss = train_epoch(model, train_loader, optimizer, criterion, device, scheduler)
        val_loss, val_f1 = eval_epoch(model, val_loader, criterion, device)

        print(f"Epoch {epoch:02d}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val F1: {val_f1:.4f}")
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), "best_model.pth")
            print(f"✅ Saved best model (F1={val_f1:.4f})")

    print(f"Training complete. Best F1: {best_f1:.4f}")


# ------------------------------------------------------------------
# 🔹 Run Training
# ------------------------------------------------------------------
# Assuming you already have:
# x_train, x_test, y_train, y_test, vocab
# and vocab contains "<PAD>" and "<UNK>"
train_model(vocab, x_train, y_train, x_val, y_val, num_classes=y_train.shape[1])


Epoch 01/50 | Train Loss: 0.5919 | Val F1: 0.1468
✅ Saved best model (F1=0.1468)


Epoch 02/50 | Train Loss: 0.5737 | Val F1: 0.1468


Epoch 03/50 | Train Loss: 0.5690 | Val F1: 0.1468


Epoch 04/50 | Train Loss: 0.5618 | Val F1: 0.2492
✅ Saved best model (F1=0.2492)


Epoch 05/50 | Train Loss: 0.5567 | Val F1: 0.2462


...

Epoch 44/50 | Train Loss: 0.0384 | Val F1: 0.4678


Epoch 45/50 | Train Loss: 0.0387 | Val F1: 0.4652


Epoch 46/50 | Train Loss: 0.0322 | Val F1: 0.4674


Epoch 47/50 | Train Loss: 0.0315 | Val F1: 0.4714


KeyboardInterrupt: 

In [39]:
# pip install optuna

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import f1_score
import optuna

# ------------------------------------------------------------------
# 🔹 Model Definition
# ------------------------------------------------------------------
class GRU_AttentionModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_layers=2, dropout=0.4):
        super(GRU_AttentionModel, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.embedding_dropout = nn.Dropout(0.4)

        self.gru = nn.GRU(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )

        # Multi-head self-attention
        self.multihead_attn = nn.MultiheadAttention(
            embed_dim=hidden_dim * 2,
            num_heads=4,
            dropout=0.3,
            batch_first=True
        )

        self.layer_norm = nn.LayerNorm(hidden_dim * 2)

        # MLP classifier head
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x, lengths=None):
        emb = self.embedding_dropout(self.embedding(x))
        packed_output, _ = self.gru(emb)
        attn_output, _ = self.multihead_attn(packed_output, packed_output, packed_output)
        attn_output = self.layer_norm(attn_output + packed_output)
        mean_output = attn_output.mean(dim=1)
        logits = self.fc(mean_output)
        return logits


# ------------------------------------------------------------------
# 🔹 Dataset
# ------------------------------------------------------------------
class TextDataset(Dataset):
    def __init__(self, X, y, vocab, max_len):
        self.X = X
        self.y = y.values
        self.vocab = vocab
        self.max_len = max_len

    def text_to_seq(self, text):
        tokens = text.split()
        seq = [self.vocab.get(tok, self.vocab["<UNK>"]) for tok in tokens[:self.max_len]]
        seq += [self.vocab["<PAD>"]] * (self.max_len - len(seq))
        return torch.tensor(seq, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.text_to_seq(self.X.iloc[idx]), torch.tensor(self.y[idx], dtype=torch.float32)


# ------------------------------------------------------------------
# 🔹 Train / Eval
# ------------------------------------------------------------------
def train_epoch(model, loader, optimizer, criterion, device, scheduler=None, clip=5):
    model.train()
    total_loss = 0
    for X_batch, y_batch in tqdm(loader, desc="Training", leave=False):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)
        optimizer.step()
        if scheduler:
            scheduler.step()
        total_loss += loss.item()
    return total_loss / len(loader)


def eval_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    preds, targets = [], []
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            logits = model(X_batch)
            loss = criterion(logits, y_batch)
            total_loss += loss.item()
            preds.append(torch.sigmoid(logits).cpu())
            targets.append(y_batch.cpu())
    preds = torch.cat(preds)
    targets = torch.cat(targets)
    f1 = f1_score(targets.numpy() > 0.5, preds.numpy() > 0.5, average="macro")
    return total_loss / len(loader), f1


# ------------------------------------------------------------------
# 🔹 Optuna Objective Function
# ------------------------------------------------------------------
def objective(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 🔸 Hyperparameters to tune
    EMBED_DIM = trial.suggest_categorical("embed_dim", [128, 256, 300])
    HIDDEN_DIM = trial.suggest_categorical("hidden_dim", [128, 256, 384])
    NUM_LAYERS = trial.suggest_int("num_layers", 1, 3)
    DROPOUT = trial.suggest_float("dropout", 0.2, 0.6)
    LR = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
    BATCH_SIZE = trial.suggest_categorical("batch_size", [32, 64, 128])

    MAX_LEN = 100
    EPOCHS = 20  # keep small for tuning speed

    # Build datasets
    train_ds = TextDataset(x_train, y_train, vocab, MAX_LEN)
    val_ds = TextDataset(x_val, y_val, vocab, MAX_LEN)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

    model = GRU_AttentionModel(len(vocab), EMBED_DIM, HIDDEN_DIM, y_train.shape[1], NUM_LAYERS, DROPOUT).to(device)
    optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)
    criterion = nn.BCEWithLogitsLoss()

    best_f1 = 0.0
    for epoch in range(EPOCHS):
        train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_f1 = eval_epoch(model, val_loader, criterion, device)
        trial.report(val_f1, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if val_f1 > best_f1:
            best_f1 = val_f1

    return best_f1


# ------------------------------------------------------------------
# 🔹 Run Optuna Tuning
# ------------------------------------------------------------------
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20, timeout=None)

print("\n🎯 Best Trial:")
trial = study.best_trial
print(f"  F1 Score: {trial.value:.4f}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# ------------------------------------------------------------------
# 🔹 (Optional) Retrain best model with tuned params
# ------------------------------------------------------------------
best_params = study.best_trial.params
print("\nRetraining with best params...")

train_model(
    vocab,
    x_train,
    y_train,
    x_val,
    y_val,
    num_classes=y_train.shape[1],
)


[I 2025-10-22 06:30:39,734] A new study created in memory with name: no-name-3198626d-5a80-43ba-87b6-7e629279e4ed
[I 2025-10-22 06:31:23,255] Trial 0 finished with value: 0.4481769298200516 and parameters: {'embed_dim': 256, 'hidden_dim': 256, 'num_layers': 1, 'dropout': 0.22593063025338786, 'lr': 0.0017149289930817563, 'batch_size': 128}. Best is trial 0 with value: 0.4481769298200516.
[I 2025-10-22 06:32:01,425] Trial 1 finished with value: 0.4755266850339118 and parameters: {'embed_dim': 300, 'hidden_dim': 128, 'num_layers': 3, 'dropout': 0.24770800542716534, 'lr': 0.00029483725734426666, 'batch_size': 32}. Best is trial 1 with value: 0.4755266850339118.
[I 2025-10-22 06:32:29,399] Trial 2 finished with value: 0.4803402155373063 and parameters: {'embed_dim': 300, 'hidden_dim': 128, 'num_layers': 2, 'dropout': 0.27394124734819464, 'lr': 0.0010634558316564187, 'batch_size': 64}. Best is trial 2 with value: 0.4803402155373063.
[I 2025-10-22 06:33:15,068] Trial 3 finished with value: 0.


🎯 Best Trial:
  F1 Score: 0.4908
  Params:
    embed_dim: 300
    hidden_dim: 128
    num_layers: 2
    dropout: 0.48683977124012534
    lr: 0.0016421109262960908
    batch_size: 128

Retraining with best params...


Epoch 01/50 | Train Loss: 0.5948 | Val F1: 0.1469
✅ Saved best model (F1=0.1469)


Epoch 02/50 | Train Loss: 0.5742 | Val F1: 0.1468


Epoch 03/50 | Train Loss: 0.5698 | Val F1: 0.1465


...

Epoch 46/50 | Train Loss: 0.0321 | Val F1: 0.4796


Epoch 47/50 | Train Loss: 0.0332 | Val F1: 0.4793


Epoch 48/50 | Train Loss: 0.0315 | Val F1: 0.4808


Epoch 49/50 | Train Loss: 0.0283 | Val F1: 0.4827


Epoch 50/50 | Train Loss: 0.0292 | Val F1: 0.4825
Training complete. Best F1: 0.4851


In [29]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 12.3 MB/s eta 0:00:00
